In [ ]:
#xml and json is used for semistructure data
#Cascade means detects motion and harcascade cant detect gender
#Standard Scaler normalize value

In [1]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files

In [4]:
# Load opencv haarcascade
haar_cascade= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

#Load pre-trainded mobilenetv2for gender and age classifier
classification_model=torch.hub.load('pytprch/vision:v0.10.0','mobilnet_v2',pretrained=True)
classification_model.eval()

# Defining label
CLASS_LABELS=['Male','Female','Kid']

# Image Preprocessing Function
def preprocess_image(image):
  img=cv2.resize(img,(224,224))
  img=img.astype(np.float32)/255.0  # Normalize the pixel value
  img=np.transpose(img,(2,0,1)) #convert to (c,h,w) represent for hog pyramid dim
  img=torch.tensor(img).unsqueeze(0) #Add batch dimension
  return img

video_path='/content/39836-424360861_small.mp4'
cap=cv2.VideoCapture(video_path)

# Get video Properties
frame_width=int(cap.get(3))
frame_height=int(cap.get(4))
fps=int(cap.get(cv2.CAP_PROP_FPS))

#Define output video writer
output_video_path='output_identified.mp4'
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
out=cv2.VideoWriter(output_video_path,fourcc,fps,(frame_width,frame_height))

# Refreshes until video is not ended
while cap.isOpened():
  ret,frame=cap.read()
  if not ret:
    break #Exit loop when video ends

  #convert frame to grayscale for Haar Cascade Deyecyion
  gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  #Detect people using haar cascade
  people=haar_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(50,100))

  for (x,y,w,h) in people:
    person_img=frame[y:y+h, x:x+w] # Crop detected person
    if person_img.size==0:
      continue
  # Preprocess image for classification
  input_tensor=preprocess_image(person_img)

  #Predict Gender/Age
  with torch.no_grad():
    outputs=classification_model(input_tensor)

  if outputs.shape[1]<len(CLASS_LABELS): #Ensure valid output
    predicted_class=torch.argmax(outputs).item()
  else:
    predicted_class=0
  label=CLASS_LABELS[predicted_class]

  #Draw bounding box and label
  color=(0,255,0) if label=='Male' else (255,0,0) if label=='Female' else (0,0,255)
  cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
  cv2.putText(frame,label,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.9,color,2)

  #Write the frame to output video
  out.write(frame)

  frame_rgb=cv2.cbtColor(frame,cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(10,6))
  plt.axis("off")
  plt.inshow(frame_rgb)
  plt.show()

# Release video capture and writer
cap.release()
out.release()

print(f'Processing complete as {output_video_path}')


files.download(output_video_path)
#cv2 has range for distance to detect the object or person

HTTPError: HTTP Error 404: Not Found